In [ ]:
!pip install pylidc

In [ ]:
!rm /root/.pylidcrc
!touch /root/.pylidcrc
!echo [dicom] >> /root/.pylidcrc
!echo path = /kaggle/input/lidcidri/LIDC-IDRI/LIDC-IDRI-0001 >> /root/.pylidcrc
!echo warn = True >> /root/.pylidcrc

In [ ]:
import os, math, glob, re
import numpy as np
import pandas as pd
import cv2

import matplotlib.pyplot as plt
import pydicom

from kaggle_datasets import KaggleDatasets

from sklearn.model_selection import train_test_split

from random import shuffle
import tensorflow as tf

import pylidc as pl
from pylidc.utils import consensus

import random
import shutil

In [ ]:
patients = pd.read_csv("../input/testdb3/LIDC-IDRI_MetaData.csv")
patients.rename(columns={"Number of images": "num_images"}, inplace=True)
patients["id"] = patients['Subject ID'].apply(lambda x : x[-4: ]).astype(int)
patients = patients.query("num_images > 50")
patients = patients.reset_index(drop=True)

In [ ]:
for i, duplicate in enumerate(patients.duplicated('Subject ID')):
    if duplicate:
        print(i, patients.iloc[i]['Subject ID'])
        patients.loc[i, 'Subject ID'] = patients.loc[i, 'Subject ID'] + "-2"

In [ ]:
for x in patients['Subject ID']:
    if x[-2:] == "-2":
        print(x)

In [ ]:
patients.head()
# patients['Subject ID'].str.contains('LIDC-IDRI-0484')
# filter = patients['Subject ID']=="LIDC-IDRI-0484-2"
# patients.where(filter, inplace = False)
# patients[(patients['Subject ID'] == 'LIDC-IDRI-0484')]
# patients.query("'Subject ID' == 'LIDC-IDRI-0484'")
# patients[patients['Subject ID'].str.contains('LIDC-IDRI-0484')]
patients[(patients['Subject ID'] == 'LIDC-IDRI-0484')]['num_images']

In [ ]:
folder = "../input/lidcidri/LIDC-IDRI/LIDC-IDRI-0001"
items = os.listdir(folder)
# patients.remove('LICENSE')
print(len(items))

for p in items:
    print(p)
    break

In [ ]:
cols2 = cols = ['patient_id', 'num_images', 'nodule', 'start', 'stop', 'num_frames']
df2 = pd.DataFrame(columns=cols2)

count = 0
for scan_id in patients['Subject ID']:
#     print(scan_id)
    if scan_id[-2:] == '-2':
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id[:-2])[1]
    else:
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id).first()
    
    nods = scan.cluster_annotations()
    


    for y, annotations in enumerate(nods):
        #     print(i, anns)
        consensus_data = consensus(annotations)[1]
        start_index, stop_index = consensus_data[2].start, consensus_data[2].stop
#         print(start_index, stop_index)

        df2.at[count, 'patient_id'] = scan_id 
        df2.at[count, 'num_images'] = patients[(patients['Subject ID'] == scan_id)]['num_images'].values[0]
        df2.at[count, 'nodule'] = y + 1
        df2.at[count, 'start'] = start_index
        df2.at[count, 'stop'] = stop_index
        df2.at[count, 'num_frames'] = stop_index - start_index + 1
        count += 1        
#         break
#     break


In [ ]:
df2.to_csv('df_nodules.csv', index=False)

In [ ]:
df2[(df2['start'] == 88) & (df2['stop'] == 91)]
# df2.query('start == 88 & stop == 91')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(patients, 
                                                  patients['Subject ID'], 
                                                  test_size=0.2, 
                                                  random_state=42,
                                                  )

In [ ]:
train_ids = y_train
val_ids = y_val

train_ids = train_ids.sort_values(ascending=True).reset_index(drop=True)
val_ids = val_ids.sort_values(ascending=True).reset_index(drop=True)
print(f"{len(train_ids)} Train Scans")
print(f"{len(val_ids)} Val Scans")

In [ ]:
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0204').first()
nods = scan.cluster_annotations()
# print(pl.utils.consensus(nods[2]))
# print(nods[2][0].bbox(pad=[(20, 20), (20, 20), (0, 0)]))
# vol = scan.to_volume()
# cmask,cbbox, masks = pl.utils.consensus(nods[2], clevel=0.5, pad=[(20, 20), (20, 20), (0, 0)])
# # pl.Annotation.visualize_in_scan(nods[2][3])
# print(cbbox)
# k = int(0.5*(cbbox[2].stop - cbbox[2].start))
# print(k)
# print(vol[cbbox].shape)

# fix, ax = plt.subplots(1, 1, figsize=(5,5))
# ax.imshow(vol[nods[2][0].bbox(pad=[(20, 20), (20, 20), (0, 0)])][:,:,2], cmap=plt.cm.gray, alpha=0.5)

In [ ]:
# for ids in train_ids:
    
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == "LIDC-IDRI-0007").first()
nods = scan.cluster_annotations()
#     if len(nods) > 1:
#         print(ids)
masks = np.zeros((140, 512, 512))
for nod in nods:
    mask = consensus(nod, pad=IMAGE_SIZE)[0].transpose(2, 0, 1)
    print(mask.shape)
    masks += mask
# plt.imshow(masks[0])
for i, m in enumerate(masks):
    if m.sum() > 0:
        print(i, m.sum())
# masks = consensus(annotations, pad=IMAGE_SIZE)[0].transpose(2, 0, 1)

In [ ]:
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == "LIDC-IDRI-0007").first()
nods = scan.cluster_annotations()

print("%s has %d nodules." % (scan, len(nods)))
# => Scan(id=1,patient_id=LIDC-IDRI-0078) has 4 nodules.

for i,nod in enumerate(nods):
    print(nod[0].bbox())
    print("Nodule %d has %d annotations." % (i+1, len(nods[i])))

In [ ]:
IMAGE_SIZE=512
# IMAGE_SIZE=None

clevel=0.5
anns = nods[0]
print(anns[0].bbox())
print(anns[0].bbox_matrix(pad=IMAGE_SIZE))
print(anns[0].bbox_matrix(pad=IMAGE_SIZE).shape)
bmats = np.array([a.bbox_matrix(pad=IMAGE_SIZE) for a in anns])
print(bmats.shape)
# print(bmats)
bmats[0]
imin,jmin,kmin = bmats[:,:,0].min(axis=0)
imax,jmax,kmax = bmats[:,:,1].max(axis=0)
print(imin,jmin,kmin)
print(imax,jmax,kmax)
# # consensus_bbox
cbbox = np.array([[imin,imax],
                  [jmin,jmax],
                  [kmin,kmax]])
print(cbbox)
masks = [a.boolean_mask(bbox=cbbox) for a in anns]
print(masks[0].shape)
# print(masks[0])
cmask = np.mean(masks, axis=0) >= .0001
print(cmask.shape)
print(cmask.transpose(2, 0, 1).shape)
cbbox = tuple(slice(cb[0], cb[1]+1, None) for cb in cbbox)
print(cbbox)
# print(masks[0].shape)
# if ret_masks:
#     return cmask, cbbox, masks
# else:
#     return cmask, cbbox


In [ ]:
plt.imshow(mask[125])

In [ ]:
mask, a = load_mask2('LIDC-IDRI-0007', 145)
print(mask.shape)
# a

In [ ]:
local_directory = "../input/lidcidri/LIDC-IDRI/LIDC-IDRI-0001"

MIN_BOUND = -1000.0
MAX_BOUND = 400.0
IMAGE_SIZE = 512

def load_dicom_img(path, img_size=512):

    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = cv2.resize(data, (img_size, img_size))
    
    return data

def load_scan(path):
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path) if s[-3:] == "dcm"]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image

PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    return image

def load_mask(scan_id, num_files):
    if scan_id[-2:] == '-2':
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id[:-2])[1]
    else:
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id).first()
    nods = scan.cluster_annotations()
    if len(nods) > 0:
        annotations = nods[0]
    
        # Create (N, 512, 512) Boolean Array with segmentation mask data
        masks = consensus(annotations, pad=IMAGE_SIZE)[0].transpose(2, 0, 1)

        # Get slice (start, end) indices of where annotations exist
        consensus_data = consensus(annotations)[1]
        start_index, stop_index = consensus_data[2].start, consensus_data[2].stop
        
    else:
        masks = np.zeros((num_files, 512, 512))
        start_index = 0
        stop_index = 0
    
    return masks, start_index, stop_index

def load_mask2(scan_id, num_files):
    if scan_id[-2:] == '-2':
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id[:-2])[1]
    else:
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id).first()
    nods = scan.cluster_annotations()
    masks = np.zeros((num_files, 512, 512))
    
    frames = []

    if len(nods) > 0:
        for nod in nods:
            annotations = nod

            # Create (N, 512, 512) Boolean Array with segmentation mask data
#             cmask, cbbox, masks2 = consensus(annotations, pad=IMAGE_SIZE)
#             print(type(cmask), type(cbbox), type(masks2))
            masks += consensus(annotations, pad=IMAGE_SIZE)[0].transpose(2, 0, 1)

            # Get slice (start, end) indices of where annotations exist
            consensus_data = consensus(annotations)[1]
            start_index, stop_index = consensus_data[2].start, consensus_data[2].stop
#             print(start_index, stop_index)      
            frames += [i for i in range(start_index, stop_index)]
    else:
        masks = np.zeros((num_files, 512, 512))
        start_index = 0
        stop_index = 0
        frames += [i for i in range(start_index, stop_index)]
    
    frames = list(set(frames))
    
    return masks, frames

def get_pixels_hu(slices, img_size=512):
    image = np.stack([cv2.resize(s.pixel_array, (img_size, img_size)) for s in slices])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approximately 0
#     image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):
        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
        
    # Make all pixel values positive
#     image += 2048
    
    return np.array(image, dtype=np.int16)

def load_dicom_scan(scan_id, img_size, img_depth, slice=False):
#     files = sorted([y for x in os.walk(f"{local_directory}/{scan_id}") for y in tf.io.gfile.glob(os.path.join(x[0], '*.dcm'))])
               #key=lambda var:[int(x) if x.isdigit() else x for x in re.findall(r'[^0-9]|[0-9]+', var)])
    scan_dir = os.path.join(local_directory, scan_id)    
    num_files = len(tf.io.gfile.glob(os.path.join(scan_dir, '*.dcm')))
    
    if scan_id[-2:] == '-2':
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id[:-2])[1]
    else:
        scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == scan_id).first()
            
    img_slices = load_scan(scan_dir)
    img3d = get_pixels_hu(img_slices)
    img3d = zero_center(normalize(img3d))

#     nodules = []
#     nods = scan.cluster_annotations()
#     for nod in nods:
#         for ann in nod:
#             nodule = img3d[ann.bbox()]
#             nodules.append(nodule)
    
    
    
#     masks, start_index, stop_index = load_mask2(scan_id, num_files)
    masks, frames = load_mask2(scan_id, num_files)

#     annotation_indices = [i for i in range(start_index, stop_index)]
    annotation_indices = frames
    random_samples_indices = None
    
    if slice==True:
        random_sample_list = [i for i in range(0, num_files) if i not in annotation_indices]
        random_samples_indices = random.sample(random_sample_list, min(len(random_sample_list), img_depth - len(annotation_indices)))
        random_samples_indices.extend(annotation_indices)
        random_samples_indices.sort()
        
#         num_files_middle = num_files//2
#         img_depth_middle = img_depth//2

#         start_depth = max(0, num_files_middle - img_depth_middle)
#         end_depth   = min(num_files, num_files_middle + img_depth_middle)
    else:
        start_depth = 0
        end_depth = num_files
                
    img3d = img3d[random_samples_indices, :, :]
    masks = masks[random_samples_indices, :, :]
#     img3d = np.stack([load_dicom_img(dicom_img, img_size=img_size) 
#                       for dicom_img in files[start_depth:end_depth]], axis=2)#.T

    if slice==True and img3d.shape[0] < img_depth:
        n_zero = np.zeros((img_depth - img3d.shape[0], img_size, img_size))
        img3d  = np.concatenate((img3d, n_zero), axis=0)
        masks = np.concatenate((masks, n_zero), axis=0)
        
    annotated_indices_new = [random_samples_indices.index(ind) for ind in annotation_indices]
        
    return img3d, masks, annotation_indices, random_samples_indices, annotated_indices_new

In [ ]:
img3d, masks, annotation_indices, random_samples_indices, annotated_indices_new = load_dicom_scan('LIDC-IDRI-0003', img_size=512, img_depth=128, slice=True)
print(img3d.shape, masks.shape, annotation_indices, random_samples_indices, annotated_indices_new)
# index = np.where(np.asarray(random_samples_indices) == 86)[0]

In [ ]:
plt.imshow(img3d[86])
np.unique(img3d[86])

In [ ]:
def display_scans(images, masks, annotated_indices):
    ncols = 4
    nrows = math.ceil(len(annotation_indices) / ncols)
    
    f, plots = plt.subplots(nrows, ncols, figsize=(40, 40))

    i = 0
    j = 0
    
    for ind in annotated_indices:
        plots[i, j].axis('off')
        plots[i, j].set_title(f"Mask Number: {ind}-{masks[ind].sum()}")
        plots[i, j].imshow(images[ind], cmap=plt.cm.gray)
        plots[i, j].imshow(masks[ind], alpha=0.5)
        j += 1
        if j % ncols == 0:
            i += 1
            j = 0
            
# index = annotated_indices_new[4]
# print(index)
# fig = plt.figure(figsize=(25, 25))
# plt.imshow(img3d[index, :, :], cmap=plt.cm.gray)
# plt.imshow(masks[index, :, :], alpha=0.25)
display_scans(img3d, masks, annotated_indices_new)

In [ ]:
plt.imshow(pydicom.read_file('../input/testdb3/LIDC-IDRI/LIDC-IDRI-0001/01-01-2000-NA-NA-30178/3000566.000000-NA-03192/1-045.dcm').pixel_array)

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_feature_list(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [ ]:
def image_example(scan_id, image, mask, annotation_indices, random_sample_indices):
    image_shape = image.shape
    
    scan_bytes = scan_id.encode("utf-8")
    img_bytes = image.tobytes()
    mask_bytes = mask.tobytes()
    
    feature = {
        'scan_id': _bytes_feature(scan_bytes),
        'height': _int64_feature(image_shape[1]),
        'width': _int64_feature(image_shape[2]),
        'num_channels': _int64_feature(image_shape[0]),
        'image': _bytes_feature(img_bytes),
        'mask' : _bytes_feature(mask_bytes),
    }
    
    annotations = tf.train.FeatureList(feature=[_int64_feature(index) for index in annotation_indices])
    sample_indices = tf.train.FeatureList(feature=[_int64_feature(index) for index in random_sample_indices])
    
    feature_list = {
        'annotation_indices': annotations,
        'random_sample_indices': sample_indices
    }
    
    return tf.train.SequenceExample(feature_lists=tf.train.FeatureLists(feature_list=feature_list), context=tf.train.Features(feature=feature))

In [ ]:
def create_tfrecord(scan_id, image, mask, annotation_indices, random_samples_indices, output_path):
    opts = tf.io.TFRecordOptions(compression_type="GZIP")
    with tf.io.TFRecordWriter(output_path, opts) as writer:
        tf_example = image_example(scan_id, image, mask, annotation_indices, random_samples_indices)
        writer.write(tf_example.SerializeToString())
    writer.close()

In [ ]:
def write_tfrecords(scan_list, output_dir, data_split, file_index, num_files):
    output_path = f"{output_dir}/{data_split}-{file_index}-{num_files}.tfrec"
    opts = tf.io.TFRecordOptions(compression_type="GZIP")
    scans = []
    for scan_id in scan_list:
        dicom_scan = load_dicom_scan(scan_id, img_size=512, img_depth=128, slice=True)
        scans.append((scan_id, dicom_scan))
        
    with tf.io.TFRecordWriter(output_path, opts) as writer:
        for scan in scans:
            scan_id, img3d, masks, annotation_indices, random_samples_indices = scan[0], scan[1][0], scan[1][1], scan[1][2], scan[1][3]
            tf_example = image_example(scan_id, img3d, masks, annotation_indices, random_samples_indices)
            writer.write(tf_example.SerializeToString())
            print("Finished writing {}".format(scan_id))
        print("Finished file {}".format(f"{data_split}-{file_index}-{num_files}.tfrec"))
    writer.close()

In [ ]:
scans_per_file = 10

output_dir = "/kaggle/working/LIDC-IDRI-TFRecords"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.mkdir(output_dir)

for i, pos in enumerate(range(0, len(train_ids), scans_per_file)):
    scan_list = [train_ids[ind] for ind in range(pos, min(len(train_ids), pos + scans_per_file))]
    write_tfrecords(scan_list, output_dir, "train", i, len(scan_list))

In [ ]:
!ls LIDC-IDRI-TFRecords/ -ltr -h

# Verify TFRecord

In [ ]:
# image_feature_description = {
#         'scan_id': _bytes_feature(scan_bytes),
#         'height': _int64_feature(image_shape[1]),
#         'width': _int64_feature(image_shape[2]),
#         'num_channels': _int64_feature(image_shape[0]),
#         'image': _bytes_feature(img_bytes),
#         'mask' : _bytes_feature(mask_bytes),
#         'annotation_indices': _int64_feature_list(annotation_indices),
#         'random_sample_indices': _int64_feature_list(random_samples_indices)
#     }

feature_description = {
    'scan_id': tf.io.FixedLenFeature([], tf.string),
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'num_channels': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string),
    'mask': tf.io.FixedLenFeature([], tf.string),
}

feature_list_description = {
    'annotation_indices': tf.io.FixedLenSequenceFeature([], tf.int64),
    'random_sample_indices': tf.io.FixedLenSequenceFeature([], tf.int64),
}

def _parse_image_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
    feature, features_list = tf.io.parse_single_sequence_example(
                            example_proto, 
                            sequence_features=feature_list_description,                                                   
                            context_features=feature_description
                        )
#     single_example = tf.io.parse_single_example(example_proto, image_feature_description)
    scan_id = feature['scan_id']
    img_height = feature['height']
    img_width = feature['width']
    num_images = feature['num_channels']
    
    img_bytes =  tf.io.decode_raw(feature['image'],out_type='int16')
    img_array = tf.reshape(img_bytes, (num_images, img_height, img_width))
   
    mask_bytes =  tf.io.decode_raw(feature['mask'],out_type='bool')
    
    mask = tf.reshape(mask_bytes, (num_images, img_height, img_width))
    
    annotation_indices = features_list['annotation_indices']
    random_sample_indices = features_list['random_sample_indices']
    
    struct = {
        'scan_id': scan_id,
        'height': img_height,
        'width': img_width,
        'num_images': num_images,
        'img': img_array,
        'mask': mask,
        'annotation_indices': annotation_indices,
        'random_sample_indices': random_sample_indices
    }
    return struct

def read_tf_dataset(storage_file_path):
    encoded_image_dataset = tf.data.TFRecordDataset(storage_file_path, compression_type="GZIP")
    parsed_image_dataset = encoded_image_dataset.map(_parse_image_function)
    return parsed_image_dataset

In [ ]:
ds = read_tf_dataset("./LIDC-IDRI-TFRecords/train-0-10.tfrec")

In [ ]:
def display_scans(images, masks, annotation_indices, all_indices):
    ncols = 3
    nrows = math.ceil(len(annotation_indices) / ncols)
    
    f, plots = plt.subplots(nrows, ncols, figsize=(25, (nrows//3) * 25))

    i = 0
    j = 0
    
    print(annotation_indices)
    print(all_indices)
    
    for ind in annotation_indices:
        corrected_index = all_indices.index(ind)
        plots[i, j].axis('off')
        plots[i, j].set_title(f"Mask Number: {ind}")
        plots[i, j].imshow(images[corrected_index], cmap=plt.cm.gray)
#         plots[i, j].imshow(masks[corrected_index], alpha=0.5)
        j += 1
        if j % ncols == 0:
            i += 1
            j = 0

In [ ]:
images = []
masks = []
annotation_indices = []
all_indices = []

for struct in ds.as_numpy_iterator():
    print(struct['scan_id'])
    images.append(struct['img'])
    masks.append(struct['mask'])
    annotation_indices.append(struct['annotation_indices'])
    all_indices.append(struct['random_sample_indices'])
#     img = struct['img']
#     print(struct)
#     print(np.unique(struct['img'][86]))
#     plt.imshow(zero_center(normalize(struct['img'][86])))
#     break

In [ ]:
scan_index = 1
display_scans(np.array(images[scan_index]), np.array(masks[scan_index]), list(annotation_indices[scan_index]), list(all_indices[scan_index]))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as manim
from skimage.measure import find_contours

import pylidc as pl
from pylidc.utils import consensus


# Query for a scan, and convert it to an array volume.
scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0004').first()
vol = scan.to_volume()

# Cluster the annotations for the scan, and grab one.
nods = scan.cluster_annotations()
anns = nods[0]

# Perform a consensus consolidation and 50% agreement level.
# We pad the slices to add context for viewing.
cmask,cbbox,masks = consensus(anns, clevel=0.5,
                              pad=[(20,20), (20,20), (0,0)])

# Get the central slice of the computed bounding box.
k = int(0.5*(cbbox[2].stop - cbbox[2].start))

# Set up the plot.
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.imshow(vol[cbbox][:,:,k], cmap=plt.cm.gray, alpha=0.5)

# Plot the annotation contours for the kth slice.
colors = ['r', 'g', 'b', 'y']
for j in range(len(masks)):
    for c in find_contours(masks[j][:,:,k].astype(float), 0.5):
        label = "Annotation %d" % (j+1)
        plt.plot(c[:,1], c[:,0], colors[j], label=label)

# Plot the 50% consensus contour for the kth slice.
for c in find_contours(cmask[:,:,k].astype(float), 0.5):
    plt.plot(c[:,1], c[:,0], '--k', label='50% Consensus')

ax.axis('off')
ax.legend()
plt.tight_layout()
#plt.savefig("../images/consensus.png", bbox_inches="tight")
plt.show()

In [ ]:
# freq = {i: [] for i in range(1, 1013)}
# for index, row in patients.iterrows():
#     freq[row.id].append(row.num_images)
# freq

In [ ]:
# sorted(temp.num_images.unique())

In [ ]:
# Train test split
# Upload small sample dataset
# Create tfrecords for image and mask

In [ ]:
IMAGE_SIZE  = 512
IMAGE_DEPTH = 64

# mri_types = ['FLAIR','T1w','T1wCE','T2w']
CHANNELS  = 1 #len(mri_types)

# local_directory = "../input/rsna-miccai-brain-tumor-radiogenomic-classification"
# local_label_path = local_directory+"/train_labels.csv"
# local_submission_path = local_directory+"/sample_submission.csv"